# PolymerConformation and CombinatorialConformation

BioCRNpyler contains a number of specialized classes to represent conformations (secondary structures) of Polymers and the reactions involved in producing these conformations. This notebook provides an introduction to these features and examples of how to use them.

__PolymerConformation__ is a Species that represents 1 or more OrderedPolymerSpecies with a set of complexes describing how these OrderedPolymerSpecies are in contact.

__CombinatorialConformation__ is a Component which wraps around PolymerConformations (containing just a single Polymer) and enumerates binding and unbinding reactions in order to form different Conformations, similarly to the CombinatorialComplex Component.

__CombinatorialConformationPromoter__ is a combination of CombinatorialConformation and a Promoter which can be used to represent the dynamic structure DNA in settings coupled to transcription.

# Example 1: PolymerConformations

In this example, we will consider an OrderedPolymerSpecies AABC and build a PolymerConformation with two internal Complexes: AB and ACS. Here, S is an external species not part of the polymer. PolymerConformations easiest to build iteratively, one Complex at a time.

In [1]:
from biocrnpyler import OrderedPolymerSpecies, PolymerConformation, Complex, Species

A, B, C, S = Species("A"), Species("B"), Species("C"), Species("S")

#First Create an PolymerConformation wrapped around a single string of Monomers (e.g. a Polymer)
#This PolymerConformation has no internal complexes
pc = PolymerConformation(polymer = [A, A, B, C])
print("pc is an emtpy PolymerConformation, which represents as an OrderedPolymerSpecies:", pc)
print("pc contains no complexes:", pc.complexes)
print("pc contains one polymer:", pc.polymers)

#Make a polymer conformation indirectly using the Complex function to join the monomers inside a Polymer
c1 = Complex([pc.polymers[0][0], pc.polymers[0][2]]) #Form a Complex A:B
print('\nc1:', c1)
pc1 = c1.parent #This complex has a parent which is a PolymerConformation
print('pc1:', pc1)
print("pc1 contains one complex:", pc1.complexes)
print("pc1 contains one polymer:", pc1.polymers)

#Next, take the polymer inside the new conformation and add a new contact which now includes an additional species S
#Notice how pc1's internal polymer is used, this way the same polymer has two contacts
c2 = Complex([pc1.polymers[0][1], pc1.polymers[0][3], S])
print("\nc2:", c2)
pc2 = c2.parent
print("pc2:", pc2)
print("pc2 contains two complexes:", pc2.complexes)
print("pc2 contains one polymer:", pc2.polymers)

#Now we produce a complex between two different polymers
#Notice that pc1 is used for one polymer and p is used for the other polymer.
#This represents the binding of two seperate polymers
c3 = Complex([pc1.polymers[0][1], pc.polymers[0][3], S])
print("\nc3:", c3)
pc3 = c3.parent
print("pc3", pc3)
print("pc3 contains two complexes:", pc3.complexes)
print("pc3 contains two polymers:", pc3.polymers)

pc is an emtpy PolymerConformation, which represents as an OrderedPolymerSpecies: ordered_polymer_A_A_B_C_
pc contains no complexes: []
pc contains one polymer: [ordered_polymer_A_A_B_C_]

c1: complex_A_B_
pc1: conformation__ordered_polymer_A_A_B_C__p0l0p0l2_complex_A_B__
pc1 contains one complex: [complex_A_B_]
pc1 contains one polymer: [ordered_polymer_A_A_B_C_]

c2: complex_A_C_S_
pc2: conformation__ordered_polymer_A_A_B_C__p0l0p0l2_complex_A_B__p0l1p0l3n_complex_A_C_S__
pc2 contains two complexes: [complex_A_B_, complex_A_C_S_]
pc2 contains one polymer: [ordered_polymer_A_A_B_C_]

c3: complex_A_C_S_
pc3 conformation__ordered_polymer_A_A_B_C__ordered_polymer_A_A_B_C__p0l0p0l2_complex_A_B__p0l1p1l3n_complex_A_C_S__
pc3 contains two complexes: [complex_A_B_, complex_A_C_S_]
pc3 contains two polymers: [ordered_polymer_A_A_B_C_, ordered_polymer_A_A_B_C_]


# Example 2: CombinatorialConformation

CombinatorialConformation is a Component which produces species and reactions allowing for the binding and unbinding of complexes in a PolymerConformation with a single internal polymer. This component transforms a set of initial_states to a set of final_states via a set of intermediate_states. Abstractly, this can be written

## Initial States $\rightleftharpoons$ Intermediate States $\rightleftharpoons$ Final States

Here $\rightleftharpoons$ denote combinatorial binding to form the different complexes in the PolymerConformation. All Excluded States found during enumeration are skipped. If no initial_states are given, the CombinatorialConformation defaults to the unbound Polymer inside the PolymerConformation in the Final States. If no intermediate states are given, initial states convert directly to final states.

Note that parameters for transitions between states can be stored under particular IDs using the state_part_ids {species:id} keyword. If None, the str(s0)-str(sf) is the default part_id for a transition between s0 and sf.

First, we consider a simple example to produce the PolymerConformation pc2 in the previous cell.

In [2]:
from biocrnpyler import CombinatorialConformation, Mixture

cc = CombinatorialConformation(final_states = [pc2])

params = {"kf":1.0, "kr":1.0}
M = Mixture(components = [cc], parameters = params)

CRN = M.compile_crn()

print(CRN.pretty_print())

geting combinations between ordered_polymer_A_A_B_C_ and conformation__ordered_polymer_A_A_B_C__p0l0p0l2_complex_A_B__p0l1p0l3n_complex_A_C_S__
computing species changes between ordered_polymer_A_A_B_C_ and conformation__ordered_polymer_A_A_B_C__p0l0p0l2_complex_A_B__p0l1p0l3n_complex_A_C_S__


UnboundLocalError: local variable 'c0' referenced before assignment

## Example 2 Continued:

The enumerated reactions can be restricted by choosing a set of intermediate states, initial states, and excluded states.

In [ ]:
from biocrnpyler import CombinatorialConformation, Mixture
params = {"kf":1.0, "kr":1.0}

#Use a different initial state
cc1 = CombinatorialConformation(initial_states = [pc1], final_states = [pc2])
M1 = Mixture(components = [cc1], parameters = params)
CRN1 = M1.compile_crn()
print("CRN starting at a different initial state:\n", CRN1.pretty_print())

#Use default initial state but contrain the intermediate states
cc2 = CombinatorialConformation(intermediate_states = [pc1], final_states = [pc2])
M2 = Mixture(components = [cc2], parameters = params)
CRN2 = M2.compile_crn()
print("CRN with contrained intermediate states:\n", CRN2.pretty_print())

#Exclude a particular state
cc3 = CombinatorialConformation(excluded_states = [pc1], final_states = [pc2])
M3 = Mixture(components = [cc3], parameters = params)
CRN3 = M2.compile_crn()
print("CRN starting at a different initial state:\n", CRN3.pretty_print())

# Example 3: CombinatorialConformationPromoter

The CombinatorialConformationPromoter is a combination of Promoter and CombinatorialConformation that can be used to model transcription. In this example, we build a simple model of the Lac Operon. This sequence has 3 Lac Repressor binding sites. The Lac Repressor Tetramer can induce looping between any two of these binding sites. Looping between binding site 1 and 3 or binding site 2 and 3 obscures the polymerase binding site and does not allow DNA to be transcribed. This is accomplished by producing the saturated looped and unlooped PolymerConformations which are used as final states. All states with lacR bound to location 3 are passed in as promoter_states with promoter_states_on = False to toggle repression of these states. By default, all other enumerated states will be transcribable.

In [ ]:
from biocrnpyler import CombinatorialConformationPromoter, Mixture, DNAassembly, SimpleTranscription, SimpleTranslation

lac_site, pol_site, lacR = Species("L"), Species("P"), Species("lacR")

#Operon without LacR Bound
FreeOperon = PolymerConformation(polymer = [lac_site, lac_site, pol_site, lac_site])

#Operons with LacR bound in 1 position
lacR0 = Complex([FreeOperon.polymers[0][0], lacR]).parent
lacR1 = Complex([FreeOperon.polymers[0][1], lacR]).parent
lacR3 = Complex([FreeOperon.polymers[0][3], lacR]).parent

#Operon with lacR bound in 2 positions (this is just used to make the 3 position case)
lacR03 = Complex([lacR3.polymers[0][0], lacR]).parent
lacR13 = Complex([lacR3.polymers[0][1], lacR]).parent
#Operon with LacR Bound in 3 positions
lacR013 = Complex([lacR03.polymers[0][1], lacR]).parent

#Operons with LacR bound in two positions and a loop
SaturatedLoop01 = Complex([lacR3.polymers[0][0], lacR3.polymers[0][1], lacR]).parent #Transcribable
SaturatedLoop03 = Complex([lacR1.polymers[0][0], lacR1.polymers[0][3], lacR]).parent #Not Transcribable
SaturatedLoop13 = Complex([lacR0.polymers[0][1], lacR0.polymers[0][3], lacR]).parent #Not Transcribable

CCP = CombinatorialConformationPromoter(
    promoter_states = [SaturatedLoop03, SaturatedLoop13, lacR3, lacR03, lacR13, lacR013],
    promoter_states_on = False,
    promoter_location = 2,
    initial_states = [FreeOperon], 
    intermediate_states = [lacR0, lacR1, lacR3], 
    final_states = [lacR013, SaturatedLoop01, SaturatedLoop03, SaturatedLoop13])

#Create a DNAassembly
A = DNAassembly(name = "lac", promoter = CCP, rbs = "rbs", protein = "betagal")

mechanisms = [SimpleTranscription(), SimpleTranslation()]

M = Mixture(components = [A], mechanisms = mechanisms, parameters = {"kf":1.0, "kr":1.0, "ktx":1.0, "ktl":1.0})

CRN = M.compile_crn()

print(CRN.pretty_print(show_rates = False))